## 작업형 제 2 유형

### ✏️ W-mart 매장의 판매 데이터를 기반으로 판매액을 예측하시오.
- 제공된 데이터 목록 : mart_train.csv(훈련 데이터), mart_test.csv(평가용 데이터)

- 예측할 컬럼 : total(총 판매액)

- 학습용 데이터(mart_train.csv)를 이용하여 총 판매액을 예측하는 모델을 만든 후 이를 평가용 데이터(mart_test.csv)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오

    - 제출 파일은 다음 1개의 컬럼을 포함해야 한다

        - pred : 예측된 총 판매액

        - 제출 파일명 : result.csv

    - 제출한 모델의 성능은 RMSE(Root Mean Sqaure Error) 평가지표에 따라 채점

In [ ]:
# 1. 문제 정의
## W-mart 매장의 판매 데이터를 활용해 총 판매액(total) 예측

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv('./data/mart_train.csv')
test = pd.read_csv('./data/mart_test.csv')

# 3. 탐색적 데이터 분석(EDA)
## 데이터 크기
print(train.shape, test.shape, '\n')

## 데이터 샘플
print(train.head(3), '\n')
print(test.head(3), '\n')

## 데이터 자료형(타입)
print(train.info())

## 기초통계량
print(train.describe(), '\n')

## object 자료형의 unique 개수
print(train.describe(include='object'), '\n')
print(test.describe(include='object'), '\n')

## 결측치 개수
print(train.isnull().sum(), '\n')
print(test.isnull().sum(), '\n')

## total 컬럼의 종류에 따른 개수 : 695개이므로 회귀 문제
print(train['total'].value_counts(), '\n')

## total 컬럼 지수표기법이 아닌 일반표기법으로 출력하여 기초통계량 확인
pd.set_option('display.float_format', '{:.10f}'.format)
print(train['total'].describe())

(700, 10) (300, 9) 

  branch       city customer_type  gender            product_line  \
0      A     Yangon        Member  Female       Health and beauty   
1      C  Naypyitaw        Normal  Female  Electronic accessories   
2      A     Yangon        Normal    Male      Home and lifestyle   

              total payment_method       rating time_of_day  day_name  
0 823457.2500000000        Ewallet 9.1000000000   afternoon  Saturday  
1 120330.0000000000           Cash 9.6000000000     morning    Friday  
2 510788.2500000001    Credit card 7.4000000000   afternoon    Sunday   

  branch       city customer_type  gender         product_line payment_method  \
0      C  Naypyitaw        Normal  Female  Fashion accessories        Ewallet   
1      B   Mandalay        Normal    Male   Food and beverages    Credit card   
2      B   Mandalay        Member  Female  Fashion accessories    Credit card   

        rating time_of_day   day_name  
0 9.6000000000   afternoon   Thursday  
1 4.300

In [8]:
# 4. 데이터 전처리
target = train.pop('total')

## 원-핫 인코딩
print(train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)

(700, 9) (300, 9)
(700, 30) (300, 30)


In [9]:
# 5. 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(560, 30) (140, 30) (560,) (140,)


In [13]:
# 6. 모델 학습 및 평가
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
!pip install xgboost
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import root_mean_squared_error as rmse     # 0 에 가까울수록 성능 좋은 모델

# from sklearn.metrics import mean_squared_error
# def rmse(y_true, y_pred):
#     return mean_squared_error(y_true, y_pred)**0.5

## 선형 회귀
lr = LinearRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_val)
print('선형 회귀 :', rmse(y_val, pred))

## 랜덤포레스트
rf = RandomForestRegressor(random_state=0)
rf.fit(X_train, y_train)
pred = rf.predict(X_val)
print('랜덤포레스트 :', rmse(y_val, pred))

## Xgboost
xg = xgb.XGBRegressor(random_state=0)
xg.fit(X_train, y_train)
pred = xg.predict(X_val)
print('xgboost :', rmse(y_val, pred))

## lightGBM
lg = lgb.LGBMRegressor(random_state=0, verbose=-1)
lg.fit(X_train, y_train)
pred = lg.predict(X_val)
print('lightGBM :', rmse(y_val, pred))

선형 회귀 : 374016.3661648522
랜덤포레스트 : 385935.56337360526
xgboost : 442570.7070483747
lightGBM : 404369.88780394377


In [15]:
# 7. 예측 및 결과 파일 생성
pred = lr.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result.csv', index=False)

print(pd.read_csv('result.csv'))     # 회귀 문제인 경우 제출 양식에 있는 샘플이 정수형이라서 소수점 처리하지 않아도 ok

                 pred
0   485484.5740277334
1   474683.4165377702
2   486669.1574532854
3   426407.8222290425
4   599942.9042138031
..                ...
295 501132.5887398273
296 554273.9221195823
297 540502.7549631428
298 550704.8821613132
299 506543.6684025062

[300 rows x 1 columns]


In [ ]:
# 8-1. 성능개선
## 원-핫 인코딩(374016.3661648522) -> 레이블 인코딩(371070.13447345496) 이므로 레이블 인코딩 채택

# 데이터 불러오기
import pandas as pd
train = pd.read_csv('./data/mart_train.csv')
test = pd.read_csv('./data/mart_test.csv')

# target 분리
target = train.pop('total')

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='object').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

# 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# 모델 학습 및 평가
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error as rmse
lr = LinearRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_val)
print(rmse(y_val, pred))

(560, 9) (140, 9) (560,) (140,)
371070.13447345496


In [20]:
# 8-2. 성능개선
## 스케일링 전(371070.13447345496) -> 스케일링 후(371070.1344734549) 차이 없으므로 생략가능

# 데이터 불러오기
import pandas as pd
train = pd.read_csv('./data/mart_train.csv')
test = pd.read_csv('./data/mart_test.csv')

# target 분리
target = train.pop('total')

# # 스케일링
# from sklearn.preprocessing import StandardScaler    # 371070.13447345496
# from sklearn.preprocessing import MinMaxScaler      # 371070.1344734549
# from sklearn.preprocessing import RobustScaler      # 371070.1344734549
# scaler = RobustScaler()
# cols = train.select_dtypes(exclude='object').columns
# train[cols] = scaler.fit_transform(train[cols])
# test[cols] = scaler.transform(test[cols])

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='object').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

# 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# 모델 학습 및 평가
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error as rmse
lr = LinearRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_val)
print(rmse(y_val, pred))

(560, 9) (140, 9) (560,) (140,)
371070.13447345496


In [21]:
# 9. 최종 제출 파일
pred = lr.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result.csv', index=False)

print(pd.read_csv('result.csv').head())

               pred
0 511234.1312016704
1 430822.6320119977
2 490155.0304542824
3 418695.6750565688
4 563949.9897618481
